<a href="https://colab.research.google.com/github/purang2/GPT/blob/main/GPT2_GreedySearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Sequence-to-Sequence Model

## Datasets for Experiments

- CNNDM
- XSum
- SAMSum
- Gigaword

In [1]:
!pip install transformers 
!pip install datasets
!pip install py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 1.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 19.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 29

In [2]:
from datasets import load_dataset

#dataset_cnndm = load_dataset("cnn_dailymail", '2.0.0') #for summarization

dataset_xsum = load_dataset("xsum")

dataset_xsum
#dataset_cnndm

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [4]:
import pandas as pd 

input_txt = "Transformer are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []

n_steps = 8
choices_per_step = 5 

with torch.no_grad():
  for _ in range(n_steps):
    iteration = dict()
    iteration["Input"] = tokenizer.decode(input_ids[0])
    output = model(input_ids = input_ids)
    #첫번째 배치의 마지막 토큰의 로짓을 선택해 소프트맥스를 적용합니다.
    next_token_logits = output.logits[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=-1)
    sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
    #가장 높은 확률의 토큰을 저장합니다.
    for choice_idx in range(choices_per_step):
      token_id = sorted_ids[choice_idx]
      token_prob  =next_token_probs[token_id].cpu().numpy() 
      token_choice = (
          f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
      ) 
      iteration[f"Choice {choice_idx+1}"] = token_choice 
    # 예측한 다음 토큰을 입력에 추가합니다.
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
    iterations.append(iteration)

pd.DataFrame(iterations)




,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformer are the,most (9.20%),only (4.13%),same (4.07%),core (3.30%),ones (2.97%)
1,Transformer are the most,common (19.33%),important (11.11%),popular (5.48%),powerful (5.08%),commonly (4.10%)
2,Transformer are the most common,types (7.73%),type (5.39%),. (4.97%),", (4.80%)",and (4.66%)
3,Transformer are the most common types,of (76.20%),. (5.01%),used (3.97%),", (3.34%)",in (1.40%)
4,Transformer are the most common types of,mod (5.03%),mods (4.12%),transformer (3.07%),trans (2.15%),Transformers (1.59%)
5,Transformer are the most common types of mod,. (13.87%),ded (11.06%),", (7.68%)",in (6.82%),ulators (6.42%)
6,Transformer are the most common types of mod.,They (21.68%),\n (12.27%),The (5.73%),There (3.06%),Most (2.65%)
7,Transformer are the most common types of mod. ...,are (30.61%),can (6.57%),'re (5.00%),add (3.25%),have (3.23%)
